# Email classification with Naive Bayes

## 1 Introduction

In this notebook, we'll introduce the theory behind the Naive Bayes learning algorithm and apply its concepts to the well known problem of classifing emails as either spam or not spam.

The content of this notebook is based on the side notes of the Machine Learning course CS229 at Standford lectured by Andrew Ng.

To a have better understading of the concepts behind the Naive Bayes algorithm, you should have familliarity with basic statics and more basic learning algorithms such as linear regression

## 2 Generative Learning Algorithms

Before going further in Naive Bayes, one must understand that this algorithm belongs to a larger class of algorithms called Generative Learning Algorithms. To make it clear, we will firstly compare it with the Discriminative Learning Algorithm class in which linear regression and logistic regression belong.

To begin our discussion, in Discriminative Learning Algorithms our goal is to learn $p(y|x)$ directly where, for instance, $y \in I\!R$ is the target variable and $x \in I\!R^d$ is the vector of features. One option to learn this relation given a training set is by finding the maximum of the log likelihood using gradient descent.

However, instead of modelling $p(y|x)$ one could model $p(x|y)$ and find the relation $p(y|x)$ indirectly by using the Bayes' rule which states: $p(y|x) = \frac{p(x|y)p(y)}{p(x)}$. The learning algorithms which use this approach are called Generative.

## 3 Naive Bayes

For this generative algorithm the features are discrete valued and our goal is to perform a classification. To simplify the understadind of how it works, we will focus on the spam filter problem in which we want to create a model that takes an email, i.e. the words in the email's body, and tell us whether this email is a spam or not. In other other words, we will deal with a text classification problem.

### 3.1 Choosing a representation

Let's say we are given a set of emails with their corresponding labels (spam or not spam). Then, we will represent an email as a vector whose length is equal to the number of distinct words considering the contents of each email in the given set, which we will call here the vocabulary.  
Our vector looks like it:
$$x = \begin{bmatrix}
1 \\
0 \\
0 \\
\vdots \\
1 \\
\end{bmatrix} =
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
\vdots \\
x_d \\
\end{bmatrix}
$$
Where the j-th represents whether the j-th word in the vocabulary. As an example, if the email contains the word "buy" and this word is the 50-th position in the vobulary, then the vector representing this email will have in its 50-th position the value 1. Besides, the target variable will read 1 when the email is a spam and 0 otherwise.

### 3.2 Modelling the problem

Now, let's build a generative model, which means choosing how to models $p(x|y)$. Since we are dealing with discrete values, we can model $p(x|y)$ with a multinomial distribution. However, if we were to do that now, each possible combination of our vector $x$ would have an associated probability, hence our model would have $2^v - 1$ parameters where $d$ is the number of words in the vocabulary, meaning our model would have too many parameters.

To solve this problem, we will make a very strong assumption on how the $x_is$ are related. That being said, we assume that the $x_is$ are conditionally independent given y. This assumption is called the Naive Bayes assumption, from where the algorithm's name. For instance, this assumption says that if $y=1$ then knowing that the word "buy" is present in the email's content has no effects on the belifs about whether the word "price" is present or not in the email's content.

With this in mind, we verify that:

$p(x_1, ..., x_v|y) = p(x_1|y)p(x_2|y, x_1)p(x_3|y, x_1, x_2)...p(x_v|y, x_1, ..., x_{v-1}) = p(x_1|y)p(x_2|y)...p(x_v|y) = \Pi_{j=1}^dp(x_j|y)$

Having said that, we now will be able to find a model with less parameters due to the Naive Bayes assumption by maximizing the joint likelihood* of the data:

*. We do not use the conditional likelihood, because the joint one provided us with some useful proporties for this case.



## 4 Applying the model

### 4.1 Pre-processing

The dataset we will use to build the model can be found at: http://www2.aueb.gr/users/ion/data/enron-spam/. The data provided there have the contents of emails separated in two folders which are "spam" and "ham". Then, we will pre process the data to put it in the format defined in the section 3.1.

In [1]:
# Useful libraries
import os
import pandas as pd
import unidecode
import random as rd
import re
from sklearn.feature_extraction.text import CountVectorizer

# Set random seed
rd.seed(9)

#### 4.1.1 Extract the data

In [2]:
nb_samples: int = 800
columns: list[str] = ["is_spam", "content"]

In [3]:
ham_df: pd.DataFrame = pd.DataFrame({
    columns[0]: pd.Series(dtype="bool"),
    columns[1]: pd.Series(dtype="int"),
})

# Go through the ham files and load the their data
ham_dir_path: str = "enron1/ham/"
for file_name in os.listdir(ham_dir_path):

    with open(ham_dir_path+file_name, "r") as f:
        try:
            email_content: str = f.read()
        except UnicodeDecodeError as e:
            continue # Ignore files that cannot be decoded for now

    row_df: pd.DataFrame = pd.DataFrame({
        columns[0]: [0.0],
        columns[1]: [email_content],
    })
    ham_df = pd.concat([ham_df, row_df], ignore_index=True, axis=0)

    if ham_df.shape[0] == nb_samples: break

print(f"Shape of the ham dataset: {ham_df.shape}")
ham_df.head()

/tmp/ipykernel_13687/3059246836.py:20: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  ham_df = pd.concat([ham_df, row_df], ignore_index=True, axis=0)


Shape of the ham dataset: (800, 2)


,is_spam,content
0,0.0,Subject: misc . questions\nhhere are some ques...
1,0.0,Subject: midlothian forecast\n- - - - - - - - ...
2,0.0,Subject: calpine daily gas nomination\n- hidal...
3,0.0,Subject: re : feedback monitor error - meter 9...
4,0.0,Subject: 98 - 1600 & 98 - 6725 needing k ' s\n...


In [4]:
spam_df: pd.DataFrame = pd.DataFrame({
    columns[0]: pd.Series(dtype="bool"),
    columns[1]: pd.Series(dtype="int"),
})

# Go through the spam files and load the their data
spam_dir_path: str = "enron1/spam/"
for file_name in os.listdir(spam_dir_path):

    with open(spam_dir_path+file_name, "r") as f:
        try:
            email_content: str = f.read()
        except UnicodeDecodeError as e:
            continue # Ignore files that cannot be decoded for now

    row_df: pd.DataFrame = pd.DataFrame({
        columns[0]: [1.0],
        columns[1]: [email_content],
    })
    spam_df = pd.concat([spam_df, row_df], ignore_index=True, axis=0)

    if spam_df.shape[0] == nb_samples: break

print(f"Shape of the spam dataset: {spam_df.shape}")
spam_df.head()

/tmp/ipykernel_13687/3935112127.py:20: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  spam_df = pd.concat([spam_df, row_df], ignore_index=True, axis=0)


Shape of the spam dataset: (800, 2)


,is_spam,content
0,1.0,"Subject: cardiology , radiology , 7 , 000 seni..."
1,1.0,Subject: hi there buddy - feel the vitality\nf...
2,1.0,Subject: \n9\nreceived : from 55 . 240 . 132 ....
3,1.0,Subject: adipren 720 - find out the latest fac...
4,1.0,"Subject: paliourg , super offers for meedicati..."


In [5]:
email_df: pd.DataFrame = pd.concat([ham_df, spam_df], axis=0, ignore_index=True)

#### 4.1.2 Clean the data

In [6]:
##### Clean the data #####

# Some words doesn't give any further information to the model since they are not related to whether an email is a spam or not
# since they will be present in both cases without distinction
# e.g. 'the', 'and', ...
words_to_ignore: list[str] = ["the", "a", "an", "and", "to", "or"]

def clean_text(row: pd.Series, words_to_ignore: list[str]) -> pd.Series:
    """
    Create a column with a list of words contained in the email's content
    Remove unuseful words
    The words are separated by the special character sequence '||'
    """

    normalized_content: str = " ".join(unidecode.unidecode(row["content"].lower()).split())
    words_to_ignore_formatted: str = f"{words_to_ignore[0]}"
    for word in words_to_ignore[1:]:
        words_to_ignore_formatted += f"{word}|"
    row["content"] = re.sub(r"[-\d:;\./,'\{\}\(\)]"+words_to_ignore_formatted, "", normalized_content)
    return row

email_df = email_df.apply(lambda row: clean_text(row=row, words_to_ignore=words_to_ignore), axis=1)
email_df.head()

,is_spam,content
0,0.0,subject: misc . questions hhere are some quest...
1,0.0,subject: midlothi fecast - - - - - - - - - - -...
2,0.0,subject: calpine daily gas nomination - hidalg...
3,0.0,subject: re : feedback monir err - meter 98413...
4,0.0,subject: 98 - 1600 & 98 - 6725 needing k ' s d...


#### 4.1.2 Tranform the data

In [7]:
vectorizer: CountVectorizer = CountVectorizer()

# Transform the features
vectorized_features = vectorizer.fit_transform(email_df["content"].values).toarray()
vectorized_features_df: pd.DataFrame = pd.DataFrame(vectorized_features, columns=vectorizer.get_feature_names_out())
vectorized_features_df.head()

,00,000,0000,000000,000000000049773,000080,0001,00018,0004,0005,...,zxklh,zxzmcnbf,zyb,zyjvit,zykfe,zyl,zynve,zyrtec,zzezrjok,zzso
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Transform the targets
targets_df: pd.DataFrame = email_df[["is_spam"]]
targets_df.head()

,is_spam
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


### 4.2 Build the model

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

#### 4.2.1 Train and test datasets

In [10]:
x_train, x_test, y_train, y_test = train_test_split(
    vectorized_features_df,
    targets_df["is_spam"],
    test_size=0.3,
    train_size=0.7,
    random_state=9,
)

#### 4.2.1 Train the model

In [11]:
multinomial_nb_model: MultinomialNB = MultinomialNB(alpha=0.005)
multinomial_nb_model.fit(x_train, y_train)

MultinomialNB(alpha=0.005)

#### 4.2.1 Evaluate the model

In [12]:
predictions = multinomial_nb_model.predict(x_test)
print(f"Accuracy: {100 * sum(predictions == y_test) / len(predictions)}%")

Accuracy: 95.41666666666667%


https://binitaregmi.medium.com/spam-classification-using-naive-bayes-algorithm-3e263061a3b0  
https://jakevdp.github.io/PythonDataScienceHandbook/05.05-naive-bayes.html  
https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html  